In [1]:
!pip install concrete-ml==1.9.0 pandas scikit-learn xgboost joblib networkx

#### ☁️ 3. CSP – Thực thi inference mã hóa
- Load Server từ Analyst
- Load ciphertext từ User.  
- Load evaluation keys
- Thực thi `model.server_run()` trên ciphertext (Concrete ML runtime).  

##### 3.1 Load Server từ Analyst

In [2]:
import zipfile
import os
from concrete import fhe
from concrete.fhe import Value

path="/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/fhe_artifacts/server.zip"

server = fhe.Server.load(path)
print("✅ Server FHE đã load thành công!")

✅ Server FHE đã load thành công!


##### 4.2 Load ciphertext từ User.    

In [3]:
import os
from concrete import fhe
from concrete.fhe import Value

path="/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/User/data_enc"

num_samples = 5
encrypted_inputs = []
for i in range(5):
    with open(f"{path}/sample_{i+1}.bin", "rb") as f:
      serialized_bytes = f.read()
    encrypted_sample = Value.deserialize(serialized_bytes)
    encrypted_inputs.append(encrypted_sample)
    print(encrypted_sample)

##### 3.3 Load evaluation keys

In [4]:
# Load evaluation keys
with open("/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/Analyst/keys/evaluation.keys", "rb") as f:
    eval_bytes = f.read()
# Giải nén từ bytes thành đối tượng EvaluationKeys
evaluation_keys = fhe.EvaluationKeys.deserialize(eval_bytes)
print("✅ Evaluation keys đã được load thành công.")

✅ Evaluation keys đã được load thành công.


##### 3.4 Thực thi `model.server_run()` trên ciphertext (Concrete ML runtime).  

In [5]:
# Thực thi model FHE trên ciphertext
results_encrypted = []
for idx, enc_value in enumerate(encrypted_inputs):
    result = server.run(enc_value, evaluation_keys=evaluation_keys)
    results_encrypted.append(result)

    # Lưu kết quả ciphertext để gửi lại cho Analyst
    os.makedirs("/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/CPS/Server/results", exist_ok=True)
    with open(f"/content/drive/MyDrive/Colab Notebooks/XGBoost-FHE-ConcreteML-Phát hiện gian lận thẻ tín dụng /Data/CPS/Server/results/result_{idx+1}.bin", "wb") as f:
        f.write(result.serialize())

print("✅ Inference FHE hoàn tất. Kết quả ciphertext đã được lưu tại /content/Server/results/")

✅ Inference FHE hoàn tất. Kết quả ciphertext đã được lưu tại /content/Server/results/
